## Deliverable 2. Create a Customer Travel Destinations Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Verkhoturye,RU,58.8627,60.8054,2.03,90,88,11.03,overcast clouds
1,1,Cape Town,ZA,-33.9258,18.4232,64.81,79,0,1.99,clear sky
2,2,Putina,PE,-15.4667,-69.4333,49.35,74,71,8.32,light rain
3,3,Taoudenni,ML,22.6783,-3.9836,68.38,25,89,9.15,overcast clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,61.29,90,78,12.30,broken clouds


In [51]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 49
What is the maximum temperature you would like for your trip? 78


In [52]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cape Town,ZA,-33.9258,18.4232,64.81,79,0,1.99,clear sky
2,2,Putina,PE,-15.4667,-69.4333,49.35,74,71,8.32,light rain
3,3,Taoudenni,ML,22.6783,-3.9836,68.38,25,89,9.15,overcast clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,61.29,90,78,12.30,broken clouds
6,6,Busselton,AU,-33.6500,115.3333,73.04,48,0,16.44,clear sky
7,7,Mataura,NZ,-46.1927,168.8643,65.48,34,0,3.33,clear sky
10,10,Ushuaia,AR,-54.8000,-68.3000,55.63,70,91,7.63,overcast clouds
14,14,Rosario Oeste,BR,-14.8361,-56.4275,75.04,96,100,3.24,light rain
16,16,Rikitea,PF,-23.1203,-134.9692,77.67,68,68,8.77,broken clouds
18,18,Sao Filipe,CV,14.8961,-24.4956,75.51,68,82,8.08,broken clouds


In [53]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cape Town,ZA,-33.9258,18.4232,64.81,79,0,1.99,clear sky
2,2,Putina,PE,-15.4667,-69.4333,49.35,74,71,8.32,light rain
3,3,Taoudenni,ML,22.6783,-3.9836,68.38,25,89,9.15,overcast clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,61.29,90,78,12.30,broken clouds
6,6,Busselton,AU,-33.6500,115.3333,73.04,48,0,16.44,clear sky


In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,64.81,clear sky,-33.9258,18.4232,
2,Putina,PE,49.35,light rain,-15.4667,-69.4333,
3,Taoudenni,ML,68.38,overcast clouds,22.6783,-3.9836,
4,Bredasdorp,ZA,61.29,broken clouds,-34.5322,20.0403,
6,Busselton,AU,73.04,clear sky,-33.6500,115.3333,
7,Mataura,NZ,65.48,clear sky,-46.1927,168.8643,
10,Ushuaia,AR,55.63,overcast clouds,-54.8000,-68.3000,
14,Rosario Oeste,BR,75.04,light rain,-14.8361,-56.4275,
16,Rikitea,PF,77.67,broken clouds,-23.1203,-134.9692,
18,Sao Filipe,CV,75.51,broken clouds,14.8961,-24.4956,


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [59]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,64.81,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
2,Putina,PE,49.35,light rain,-15.4667,-69.4333,Hotel Isla Suasi
4,Bredasdorp,ZA,61.29,broken clouds,-34.5322,20.0403,Bredasdorp Country Manor
6,Busselton,AU,73.04,clear sky,-33.6500,115.3333,Observatory Guest House
7,Mataura,NZ,65.48,clear sky,-46.1927,168.8643,Ellie's Villa
10,Ushuaia,AR,55.63,overcast clouds,-54.8000,-68.3000,Albatros Hotel
14,Rosario Oeste,BR,75.04,light rain,-14.8361,-56.4275,Alvorada Palace accomodation
16,Rikitea,PF,77.67,broken clouds,-23.1203,-134.9692,People ThankYou
18,Sao Filipe,CV,75.51,broken clouds,14.8961,-24.4956,Hotel Xaguate
19,Saint George,US,57.25,broken clouds,37.1041,-113.5841,Best Western Plus Abbey Inn


In [60]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#from config import g_key
#gmaps.configure(api_key=g_key)
#gmaps.figure()